In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Configuración inicial
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 128
learning_rate = 0.001
epochs = 50
size_cube = 3
k_prev = 1

num_peaks = 15
input_size = (size_cube**3)*num_peaks + 3*k_prev  # 408
output_size = 3

# Función para cargar datos
def load_dataset(data_dir):
    data = np.load(os.path.join(data_dir, 'data_1000000_n3_k1.npz'))
    return data['inputs'], data['outputs']

# Cargar y preprocesar datos
X, y = load_dataset('/home/riemann007/JupyterLab/Tesis/Proyecto/Datos/Entrenamiento/Vecindades')
X_mean, X_std = X.mean(axis=0), X.std(axis=0)
np.save("X_mean", X_mean)
np.save("X_std", X_std)
X = (X - X_mean) / (X_std + 1e-8)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True)

train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
val_dataset = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Definición del modelo
class TractographyMLP(nn.Module):
    def __init__(self):
        super(TractographyMLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),

            nn.Linear(128, output_size)
        )

    def forward(self, x):
        return self.layers(x)

model = TractographyMLP().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CosineEmbeddingLoss()

# Funciones auxiliares
def train(model, dataloader):
    model.train()
    running_loss = 0.0
    for inputs, targets in dataloader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, targets, torch.ones(targets.size(0)).to(device))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    return running_loss / len(dataloader)

def validate(model, dataloader):
    model.eval()
    running_loss = 0.0
    angular_errors = []

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets, torch.ones(targets.size(0)).to(device))

            cos_sim = nn.functional.cosine_similarity(outputs, targets)
            angular_error = torch.acos(torch.clamp(cos_sim, -1, 1)) * (180 / np.pi)
            angular_errors.extend(angular_error.cpu().numpy())

            running_loss += loss.item()

    return running_loss / len(dataloader), np.mean(angular_errors)

# --- Entrenamiento con almacenamiento de métricas ---
train_losses, val_losses, val_angles = [], [], []
best_val_loss = float('inf')

for epoch in range(epochs):
    train_loss = train(model, train_loader)
    val_loss, val_angle = validate(model, val_loader)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_angles.append(val_angle)

    print(f'Epoch {epoch+1}/{epochs}')
    print(f'Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Angle: {val_angle:.2f}°')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        print('Model saved!')

print('Entrenamiento completado!')


Epoch 1/50
Train Loss: 0.0715 | Val Loss: 0.0494 | Val Angle: 16.10°
Model saved!
Epoch 2/50
Train Loss: 0.0543 | Val Loss: 0.0459 | Val Angle: 15.51°
Model saved!
Epoch 3/50
Train Loss: 0.0508 | Val Loss: 0.0445 | Val Angle: 15.26°
Model saved!
Epoch 4/50
Train Loss: 0.0490 | Val Loss: 0.0435 | Val Angle: 15.06°
Model saved!
Epoch 5/50
Train Loss: 0.0477 | Val Loss: 0.0429 | Val Angle: 14.96°
Model saved!
Epoch 6/50
Train Loss: 0.0468 | Val Loss: 0.0423 | Val Angle: 14.85°
Model saved!
Epoch 7/50
Train Loss: 0.0461 | Val Loss: 0.0422 | Val Angle: 14.81°
Model saved!
Epoch 8/50
Train Loss: 0.0455 | Val Loss: 0.0417 | Val Angle: 14.73°
Model saved!
Epoch 9/50
Train Loss: 0.0450 | Val Loss: 0.0415 | Val Angle: 14.69°
Model saved!
Epoch 10/50
Train Loss: 0.0445 | Val Loss: 0.0414 | Val Angle: 14.66°
Model saved!
Epoch 11/50
Train Loss: 0.0442 | Val Loss: 0.0413 | Val Angle: 14.64°
Model saved!
Epoch 12/50
Train Loss: 0.0439 | Val Loss: 0.0409 | Val Angle: 14.59°
Model saved!
Epoch 13/50
T

In [ ]:
# --- Generación y guardado de gráficas ---
epochs_range = list(range(1, epochs+1))

# Gráfica 1: Train vs Validation Loss
plt.figure(figsize=(8,6))
plt.plot(epochs_range, train_losses, label="Train Loss")
plt.plot(epochs_range, val_losses, label="Validation Loss")
plt.xlabel("Época")
plt.ylabel("Loss")
plt.title("Evolución de la pérdida durante el entrenamiento")
plt.legend()
plt.grid(True)
plt.savefig("loss_curve.png", dpi=300)
plt.show()
plt.close()

# Gráfica 2: Validation Angular Error
plt.figure(figsize=(8,6))
plt.plot(epochs_range, val_angles, color="orange", label="Validation Angular Error")
plt.xlabel("Época")
plt.ylabel("Error Angular (°)")
plt.title("Error angular de validación durante el entrenamiento")
plt.legend()
plt.grid(True)
plt.savefig("val_angle_curve.png", dpi=300)
plt.show()
plt.close()

print("Gráficas guardadas en 'loss_curve.png' y 'val_angle_curve.png'")